In [1]:
# Spectral clustering uses the eigenvectors of a Laplacian of the input
# graph to find a given number of clusters which satisfy a given
# metric.
# Balanced Cut is such a metric, to perform spectral clustering using 
# the balanced cut metric in cugraph we use:
# cugraph.spectralBalancedCutClustering(G, num_clusters, num_eigen_vects) 
#   G: A cugraph.Graph object
#   num_clusters: The number of clusters to find
#   num_eig: (optional) The number of eigenvectors to use

In [2]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from scipy.io import mmread

In [3]:
# Read in the data file into scipy matrix format
mmFile='/datasets/networks/karate.mtx'
M = mmread(mmFile).asfptype().tolil()
M = M.tocsr()

In [4]:
# Load the structure of the graph into GPU memory and create a CuGraph
# graph object:
row_offsets = cudf.Series(M.indptr)
col_indices = cudf.Series(M.indices)
values = cudf.Series(M.data)
G = cugraph.Graph()
G.add_adj_list(row_offsets, col_indices, values)

In [5]:
# Call spectralBalancedCutClustering on the graph for 3 clusters
# using 3 eigenvectors:
df = cugraph.spectralBalancedCutClustering(G, 3, num_eigen_vects=3)

In [6]:
# Check the edge cut score for the produced clustering
score = cugraph.analyzeClustering_edge_cut(G, 2, df['cluster'])
score

9.5

In [7]:
# See which nodes are in cluster 0:
part = []
for i in range(len(df)):
    if (df['cluster'][i] == 0):
        part.append(df['vertex'][i])
print(part)

[9, 14, 15, 18, 20, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33]
